In [1]:
# pip install mysql-connector-python

In [1]:
from preamble import *

In [2]:
import mysql.connector

In [ ]:
db_config = {
  'host': '10.125.121.226',  # MySQL 서버 IP 주소 또는 호스트 이름
  'database': 'pms_db_dev_gs',    # 데이터베이스 이름
  'user': 'opti',            # 사용자 계정 이름
  'password': 'tiger'      # 비밀번호
}
try :
  # MySQL 서버 연결
  conn = mysql.connector.connect(**db_config)
  cursor = conn.cursor()

  # 2. SQL 쿼리 실행 및 데이터 추출
  query = "SELECT * FROM tb_motor"
  df = pd.read_sql_query(query, conn)

except mysql.connector.Error as e:
  print("MySQL 연결 또는 쿼리 실행 오류:", e)

finally:
  if conn and conn.is_connected():
    cursor.close()
    conn.close()
    print("MySQL 연결이 종료되었습니다.")

In [4]:
pms_data = df.dropna()

In [5]:
pms_data.shape

(87719, 7)

In [6]:
pms_data.head()

,motor_id,equipment_id,center_id,channel_id,acq_date,data_array,proc_stat
384,motor_n_1,cDAQ9189-213AD12Mod1/CHN0,gosan_n,CHN0,2024-05-04 14:47:13,H4sIAAAAAAAAAIR9CdIsSW8bj+IDeByVVVnbAXQK3f8att...,2
385,motor_n_1,cDAQ9189-213AD12Mod1/CHN0,gosan_n,CHN0,2024-05-04 15:17:13,H4sIAAAAAAAAAIS9idHsSI8kCFFGgKk1khG8BBgpRn81dv...,2
386,motor_n_1,cDAQ9189-213AD12Mod1/CHN0,gosan_n,CHN0,2024-05-04 15:47:14,H4sIAAAAAAAAAIy9CbIsOY4kiKPMASZajEZbD1Cn6Ptfo2...,2
387,motor_n_1,cDAQ9189-213AD12Mod1/CHN0,gosan_n,CHN0,2024-05-04 16:17:14,H4sIAAAAAAAAAIR9C7LkQI4bj+IDuB1KSanPAXwK3/8atr...,2
388,motor_n_1,cDAQ9189-213AD12Mod1/CHN0,gosan_n,CHN0,2024-05-04 16:47:15,H4sIAAAAAAAAAIx9ixHkSK4cTJEB2hcs/oo0QFbIfzckzX...,2


In [7]:
import base64
import gzip
import json
import numpy as np

# 데이터 압축 및 압축 해제 함수
def decompress(data):
  try:
    byte_data = gzip.decompress(base64.standard_b64decode(data))
    temp = byte_data.decode("utf-16")
    temp_jarr = json.loads(temp)
    return np.array(list(map(lambda x: float(x), temp_jarr)))
  except Exception as e:
    logger.error(f"Data decompression error: {e}")
    raise

In [8]:
pms_data['data_array'] = pms_data['data_array'].apply(decompress)

print(pms_data.head()) 

      motor_id               equipment_id center_id channel_id  \
384  motor_n_1  cDAQ9189-213AD12Mod1/CHN0   gosan_n       CHN0   
385  motor_n_1  cDAQ9189-213AD12Mod1/CHN0   gosan_n       CHN0   
386  motor_n_1  cDAQ9189-213AD12Mod1/CHN0   gosan_n       CHN0   
387  motor_n_1  cDAQ9189-213AD12Mod1/CHN0   gosan_n       CHN0   
388  motor_n_1  cDAQ9189-213AD12Mod1/CHN0   gosan_n       CHN0   

               acq_date                                         data_array  \
384 2024-05-04 14:47:13  [-0.146385, -0.269267, 0.199353, 0.195184, 0.0...   
385 2024-05-04 15:17:13  [0.141742, 0.117594, 0.043271, -0.021963, -0.3...   
386 2024-05-04 15:47:14  [0.711275, 0.371583, -0.057603, 0.538894, 1.00...   
387 2024-05-04 16:17:14  [0.441287, 0.516323, 0.785072, 0.232665, 0.998...   
388 2024-05-04 16:47:15  [-0.031127, -0.164337, -0.233552, 0.049055, -0...   

     proc_stat  
384          2  
385          2  
386          2  
387          2  
388          2  


C:\Users\user\AppData\Local\Temp\ipykernel_18808\2872536943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pms_data['data_array'] = pms_data['data_array'].apply(decompress)


In [9]:
pms_data.to_csv(path_or_buf='data/pms_data_decompressed.csv', index=False)

In [12]:
zero_data = pms_data[pms_data['proc_stat'] == 0]
zero_data.shape

(70, 7)